# Lettuce: A Language With Let Bindings

We will now introduce our first language called _lettuce_, a language with let bindings. This language is inspired by an existing family of languages called `ML` (Meta Language), which has been implemented in general purpose langauges such as `SML`, `OCAML`, `ELM` and so on. The Lettuce language will illustrate the following important concepts:

- Immutable data: environments
- Function calls, closures and recursive functions.
- Mutable data through references.

Read more about ML family of languages here : https://en.wikipedia.org/wiki/ML_(programming_language)

Let's build some example programs to understand the central concept in our let language: the _let binding_.

## Let Bindings

Let bindings allow us to bind an identifier to a value and use it inside a body expression, using the syntax of the following form

`let (symbolName) = (defining expression) in (body expression)`

_Let's_  practice writing programs in Lettuce (no pun intended).

#### Example 1

~~~
let x = 3.5 in x + x
~~~

First of all, it defines a symbol named `x`. The defining expression here is `3.5`, and thus evaluates to `3.5`.
The body expression is `x + x`. With `x` bound to `3.5`, what does this evaluate to? The answer is `7.0`.

Therefore the whole expression can be successively rewritten as follows:

__eval__ ( "let x = 3.5 in x + x") -->  __eval__( 3.5 + 3.5 ) --> __eval__ (7.0) -->  7.0


#### Example 2

We can now nest let bindings  as follows:

~~~
let x = 3.0 in 
    let y = 4.5 - x in 
        x + 2.0 * y
~~~

This has two let bindings:

Outer binding:

~~~
let x = 3.0 in ((body expression # 1))
~~~

defines the symbol `x` to the defining expression `3.0` and the body
expression \# 1 is given by 


~~~
let y = 4.5 in x + 2.0 * y
~~~

defines the symbol `y` to the defining expression `4.5 - x` and the body expression \# 2 is
given by ` x + 2.0 * y`.

Now that we have picked it apart, let us evaluate this expression ourselves.


__eval__("let x = 3.0 in let y = 4.5 - x in x + 2.0 * y") --> __eval__ (" let y = 4.5 - 3.0 in 3.0 + 2.0 * y " ) 
 -->  __eval__("let y = 1.5 in 3.0 + 2.0 * y") --> __eval__ (3.0 + 2.0 * 1.5) --> __eval__(6.0) --> 6.0 


#### Example 3

Let us do one more with a different nesting of let bindings.

~~~
let x = ( let y = 5 in y * y )  in  
      let z = 15 * x in 
               x - z 
~~~

Thus far we saw body expressions having let bindings. Now we can allow let bindings for the defining expressions as well. 

~~~
let x = (defining expression # 1) in (body expression # 1)
~~~

Defining expression \# 1 itself is

~~~
let y = 5 in y * y
~~~

The body expression \# 1 is 

~~~
let z = 15 * x in x - z
~~~


How do we evaluate it? Now is a good time to break up the evaluation of a let binding 

~~~
let sym = (defining expression) in (body expression)
~~~

as follows: 

1. Evaluate the defining expression : __eval__ (defining expression)
2. Associate the symbol `sym` with the result from step 1.
3. Evaluate the body expression but now remember the association from step 1.


## Let Bindings in Scala

Scala does not have let bindings with the same syntax but it has
let bindings through the `val` declaration.

~~~
val x = 10
val y = x + 10
val z = y + 10
(x + y + z)
~~~

Here we have assigned `x` to 10 and `y` to `x+10` and then `z` to `y + 10` with `x+y+z` being
the overall value of our code fragment above.

Compare with how we could write something similar in Lettuce

~~~
let x = 10 in 
    let y = x + 10 in 
        let z = y + 10 in 
             x + y + z
~~~


Here is another example in Scala:

~~~
val z = { 
    val x = 10 
    val y = 15
    x + y
 }
return z + 10
~~~

In Lettuce, we would write

~~~
let z = ( 
         let x = 10 in 
             let y = 15 in 
                 x + y 
        ) 
    in 
     z + 10
~~~

Now you appreciate how let bindings also arise in Scala without the `let id = defining-expr in body-expr` syntax.


## Scoping in Lettuce

It is now important to understand scoping rules. Scoping rules allow us to understand which let binding 
applies to a given identifier `x`.

Consider the following program

~~~
let y = 15 in 
    let x = 10 in x + y  
~~~

Consider the expression `x + y`. You can understand which let binding has defined `x` an which let binding has
defined `y`.

However, consider the more complicated case:

~~~
let x = 10 in 
let x = x + 10 in 
let x = x + 10 in 
    x + 10
~~~

Here we are repeatedly redefining `x` however the right hand sides refer back the left hand side. Is this allowed?
The equivalent code in scala raises an error since a statement like `val x = x + 10` in scala is forbidden.


In [0]:
val x = 10
val x = x + 10 
val x = x + 10 
x + 10

cmd0.sc:2: x is already defined as value x
val x = x + 10 
          ^cmd0.sc:3: x is already defined as value x
val x = x + 10 
          ^Compilation Failed

: 

However, this is a matter of definition. In Lettuce, we could pretend that the scoping rules allow the defining expression in a let binding to be evaluated first under previously defined let bindings.

Therefore, the code fragment 

~~~
let x = 10 in 
   let x = x + 10 in 
      let x = x + 10 in 
         x + 10
~~~

is entirely equivalent to 

~~~
let x0 = 10 in 
    let x1 = x0 + 10 in 
        let x2 = x1 + 10 in 
            x2 + 10
~~~


How about this one?

~~~
let y = 15 in 
     let x = ( let y = 10 in y + y ) in 
         y + x 
~~~

There are two places where y is being defined. So which binding of `y` matters for `y + x`? 

Answer is that this is entirely equal to 

~~~
let y1 = 15 in 
     let x1 = ( let y2 = 10 in y2 + y2) in 
         y1 + x1 
~~~

Thus the value of `y` is `15` when evaluating `y + x` but equals `10` when evaluating `y + y`.

Scoping is not intuitive but very important to understand.

The same holds in scala.

~~~
val y = 15
val x = { 
       val y = 10
       y + y
    }
x + y
~~~

In [1]:
val y = 15
val x = { 
       val y = 10 // this overshadows the val y = 15 declaration
       y + y
    } // the scope for val y = 10 on line 3 ends here, the val y = 15 declaration comes back in scope.
print(s"y = $y, x + y = ${x+y}")

y = 15, x + y = 35

y: Int = 15
x: Int = 20

## Basic Types in Lettuce

To begin with, we will allow variables in Lettuce to be double precision numbers and Booleans.   Later, of course, we will add more types to the language. This means that we will allow the entire grammar of expressions and conditional expressions we have previously defined into Lettuce.

~~~
let x = 25 in 
   let y = exp(x) in 
       let z = (x >= y) in 
            if (z) 
             then y 
             else x
~~~

Here `x` is defined as a double precision value `25` and `y` computes the exponential of `x`. Finally, `z` is a Boolean that computes `true` if `x >= y` and `false` otherwise. Finally, the entire expression evaluates to `y` if `z` is true otherwise to x


## Function Calls in Lettuce

We will allow definitions of functions and calls to them in Lettuce. All functions in Lettuce can have just one argument to them.

~~~
let w = 3.1415 in 
let f = function (x)  
          let y = 2 * x - 5 in 
          let z = 2 * w * x in 
              y * sin(z)
        in 
        w * w + f(1)
~~~

Here we have defined a function `f` using let bindings.

~~~
let <function name> = function (<formal parameter name>) 
                           <body expression for function>
                       in 
                           <body expression>
~~~

For the example above, the `<function name>` is `f`, the `<formal parameter name>` is `x`, 
the `<body expression for function>` is 
~~~
let y = 2 * x - 5 in 
    let z = 2 * w * x in 
        y * sin(z)
~~~
the other `<body expression>` is ` w * w + f(1)`.

It is important to understand scoping rules for a function call. First, the variable `x` inside the body of `f` 
refers to its formal parameter. What about the variable `w` inside the body of `f`? It must refer to the
let binding `let w = 3.1415 in ` that immediately preceds it. If we made this choice, it is called
_static scoping_. I.e., variables inside a function are resolved at the time of definition of the function.
A different choice can be made called _dynamic scoping_ would resolve the variable not at the time when
`f` is resolved but when `f` is actually called.

For instance, consider

~~~
let w = 3.1415 in 
let f = function (x) 
        x * sin(2 * w * x) 
    in 
let w = 3.1415/2.0 in 
    w * w + f(1)
~~~

Under static scoping, the `w` inside the body of function `f` resolves to `3.1415`, whereas the `w` 
at the call to f `w * w + f(1)` resolves to `3.1415/2.0`. 

However, under dynamic scoping, the value of `w` inside the function `f` is bound to whatever value it takes
when `f` is called. Thus, it would be bound to `3.1415/2.0` since that is what `w` is when we call `f(1)`.
If you think of dynamic scoping as "counter intuitive", just think again. It is the scoping that python uses.

~~~
# Here is a python program
w = 25
def f(x):
   return x + w
w = 50
print(f(30)) # Does it print 55 or 80?
~~~

What kind of scoping does scala have? Can we try? First of all, scala does not permit redefining a `val` inside the same scope.  The program below will raise a syntax error (try it).

~~~
val w = 25
def f(x: Int): Int = x + w
val w = 50
val z = f(30)
~~~

But we can run a program like this and conclude that scala does static scoping as well.
  

In [2]:
val w = 10
def f(x: Int): Int = { print(s"From inside : w = $w"); x + w  }
val z = {
    val w = 20
    f(20)
}


From inside : w = 10

w: Int = 10
defined function f
z: Int = 30

## Currying Functions

Lettuce has just one argument functions. How do we do multiple argument funcitons in Lettuce? Simple, we can always use currying to define a function with multiple arguments.

~~~
let f = function (x) function (y) x + y in 
   f (10) (20)
~~~

This defines a function of two arguments `f` as first a function that takes in `x` and returns
a function that takes in `y` and adds ` x + y`.

An equivalent way of writing would be

~~~
let f = function (x) 
        let g = function (y) x + y in 
            g
        in 
   let y1 = f(10)  in 
         y1(20)
~~~

Currying is also supported in Scala. Let us take an example.

In [3]:
def f (x: Int) (y: Int) = x + y

val v1: Int => Int = f(10)

val v2 = v1(20)

defined function f
v1: Int => Int = ammonite.$sess.cmd2$Helper$$Lambda$2135/0x0000000801605040@30726488
v2: Int = 30

Currying is very useful in the following situation: 
- Apply some operations to your inputs and
- Call a user defined function on the result.


In [4]:
def multiplyAndProcess(x: Int, y:Int) (f: Int => Int) = {
    f ( x* y)
}

val v1 = multiplyAndProcess(20, 10) { x => 2 * x }

defined function multiplyAndProcess
v1: Int = 400

## Grammar for Lettuce

We are now ready to define a grammar for Lettuce.

$$\begin{array}{rcll}
\mathbf{Program} & \rightarrow & TopLevel(\mathbf{Expr}) \\[5pt]
\mathbf{Expr} & \rightarrow & Const(\mathbf{Number}) \\
 & | & True \\
 & | & False \\
 & | & Ident(\mathbf{Identifier}) \\
 & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Minus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Mult (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Div (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Log (\mathbf{Expr}) \\
 & | & Exp (\mathbf{Expr}) \\
 & | & Sine (\mathbf{Expr}) \\
 & | & Cosine (\mathbf{Expr}) \\
 & | & Geq (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Eq (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & And ( \mathbf{Expr}, \mathbf{Expr} ) \\
 & | & Or ( \mathbf{Expr}, \mathbf{Expr} ) \\
 & | & Not ( \mathbf{Expr}) \\
 & | & IfThenElse(\mathbf{Expr}, \mathbf{Expr}, \mathbf{Expr}) & \text{if (expr) then expr else expr} \\
 & | & Let( \mathbf{Identifier}, \mathbf{Expr}, \mathbf{Expr}) & \text{let identifier = expr in expr} \\
 & | & FunDef( \mathbf{Identifier}, \mathbf{Expr}) & \text{function (identifier) expr } \\ 
 & | & FunCall(\mathbf{Expr}, \mathbf{Expr}) & \text{function call - identifier(expr)} \\[5pt]
\end{array}$$


It helps to how programs written in the concrete syntax translate into abstract syntax using the grammar.

### Example 1

~~~
let x = 10 + 15 in 
    let y = x >= 25 in 
        if (y) 
        then x 
        else x - 35
~~~

will translate to 

~~~
Let("x", Plus(Const(10), Const(15)), 
         Let("y", Geq(Ident("x"), Const(25)), 
                IfThenElse( Ident("y"), 
                               Ident("x"), 
                               Minus(Ident("x"), Const(35))
                           )
             )
    )
~~~

### Example 2

~~~
let square = function (w)  w * w in 
    25 + square(25)
~~~

will translate to 

~~~
Let("square", FunDef("w", Mult(Ident("w"), Ident("w")) ),
            Plus(Const(25), FunCall(Ident("square"), Const(25)))
~~~
           

In [5]:
sealed trait Program
sealed trait Expr

case class TopLevel(e: Expr) extends Program

case class Const(v: Double) extends Expr // Expr -> Const(v)
case object True extends Expr // Expr -> True
case object False extends Expr // Expr -> False
case class Ident(s: String) extends Expr // Expr -> Ident(s)

// Arithmetic Expressions
case class Plus(e1: Expr, e2: Expr) extends Expr // Expr -> Plus(Expr, Expr)
case class Minus(e1: Expr, e2: Expr) extends Expr // Expr -> Minus(Expr, Expr)
case class Mult(e1: Expr, e2: Expr) extends Expr // Expr -> Mult (Expr, Expr)
case class Div(e1: Expr, e2: Expr) extends Expr // Expr -> Mult(Expr, Expr)
case class Log(e: Expr) extends Expr 
case class Exp(e: Expr) extends Expr
case class Sine(e: Expr) extends Expr
case class Cosine(e: Expr) extends Expr

// Boolean Expressions
case class Geq(e1: Expr, e2:Expr) extends Expr
case class Eq(e1: Expr, e2: Expr) extends Expr
case class And(e1: Expr, e2: Expr) extends Expr
case class Or(e1: Expr, e2: Expr) extends Expr
case class Not(e: Expr) extends Expr

//If then else
case class IfThenElse(e: Expr, eIf: Expr, eElse: Expr) extends Expr

//Let bindings
case class Let(s: String, defExpr: Expr, bodyExpr: Expr) extends Expr

//Function definition
case class FunDef(param: String, bodyExpr: Expr) extends Expr

// Function call
case class FunCall(funCalled: Expr, argExpr: Expr) extends Expr




defined trait Program
defined trait Expr
defined class TopLevel
defined class Const
defined object True
defined object False
defined class Ident
defined class Plus
defined class Minus
defined class Mult
defined class Div
defined class Log
defined class Exp
defined class Sine
defined class Cosine
defined class Geq
defined class Eq
defined class And
defined class Or
defined class Not
defined class IfThenElse
defined class Let
defined class FunDef
defined class FunCall

In [6]:
val p1 = TopLevel(Let("x", Plus(Const(10), Const(15)), 
         Let("y", Geq(Ident("x"), Const(25)), 
                IfThenElse( Ident("y"), 
                               Ident("x"), 
                               Minus(Ident("x"), Const(35))
                           )
             )
    ))

p1: TopLevel = TopLevel(
  Let(
    "x",
    Plus(Const(10.0), Const(15.0)),
    Let(
      "y",
      Geq(Ident("x"), Const(25.0)),
      IfThenElse(Ident("y"), Ident("x"), Minus(Ident("x"), Const(35.0)))
    )
  )
)

In [7]:
val p2 = TopLevel(Let("square", FunDef("w", Mult(Ident("w"), Ident("w")) ), Plus(Const(25), FunCall(Ident("square"), Const(25)))))

p2: TopLevel = TopLevel(
  Let(
    "square",
    FunDef("w", Mult(Ident("w"), Ident("w"))),
    Plus(Const(25.0), FunCall(Ident("square"), Const(25.0)))
  )
)

## Well Formed vs. Ill Formed Programs

You may already note that the abstract syntax allows us to write all kinds of programs that should be clearly illegal. Here are some examples:

~~~
let x = y + z in y * z
~~~

`y` and `z` are undeclared at the time we are binding `y+z` to `x`, same problem when we evaluate `y * z`.

~~~
let y = 10 in 
let z = 15 in 
let x = y + (z >= 25) in 
    false
~~~

We are trying to add a number `y` and a boolean expression `z >= 25` what is the meaning?

~~~
let y = 15 in 
let z = 25 + function (w) w * w in 
    y(31)
~~~

Wow! where to begin? first of all, we are adding 25 to a function definition `function (w) w * w` and
we are calling `y(31)` when `y` is not even a function.

Programs like these are really confusing. Do we even allow them? Why can't we just forbid them in the abstract syntax tree?

__Fact:__ Abstract Syntax Trees are built from Context Free Grammars. Unfortunately, due to limitations arising
from context freedom (i.e, LHS of each rule is just a single nonterminal), we cannot really enforce rules like 
- Variables should be declared before use.
- Boolean expressions cannot be added to integer expressions
- Function calls can only be made over proper functions.

Programming languages generally have different enforcement mechanisms:
- Check if a program is _well-formed_ while parsing the program.
- Check types of expressions while parsing the program (static type checking).
- Check types of expressions while interpreting/evaluating the program (runtime typechecking).
- A combination of above.

Let us use scala as an example. Certain issues are caught at compile time.


In [7]:
//Adding a number to a boolean, scala will catch it right away
def f(x: Int): Int = { x + (x >= 233) }


cmd7.sc:1: overloaded method value + with alternatives:
  (x: Int)Int <and>
  (x: Char)Int <and>
  (x: Short)Int <and>
  (x: Byte)Int
 cannot be applied to (Boolean)
def f(x: Int): Int = { x + (x >= 233) }
                         ^Compilation Failed

: 

In [7]:
//Comparing a string to a number?
def comp(x: String): Boolean = x >= 233

cmd7.sc:1: type mismatch;
 found   : Int(233)
 required: String
def comp(x: String): Boolean = x >= 233
                                    ^Compilation Failed

: 

In [7]:
// Trying to call a number?
def nonsence(x: Int): Int = x(355)

cmd7.sc:1: Int does not take parameters
def nonsence(x: Int): Int = x(355)
                             ^Compilation Failed

: 

In [8]:
def incompleteCase(x: (Int, Int, Int)): Int = x match {
    case (i, j, k) if i == 0 => j - k
    case (i, j, k) if j > 0 => i - k
    case (i, j, k) if k > 0 => j - i
    // Clearly more cases are needed here. 
    // Will scala complain when compiling this?
}

defined function incompleteCase

In [9]:
// RUNTIME CHECKING : scala will complain when running
print(incompleteCase( (0, 2, -5) ))
incompleteCase( (2, -3, -5) )

7

: 

In that same vein, we can catch errors in Lettuce programs at compile time. 
- Variables should be declared before use (we can catch this using a special declare before use check).
- Boolean expressions cannot be added to integer expressions (we will catch this when we do type checking).
- Function calls can only be made over proper functions (we will catch this during type checking).

## Example: Checking Whether Variables Are Declared Before Usage.

Let us define inference rules that will help us check if an expression is checked before use. To do so, we will define a set of currently declared variables that are in scope called $S$. 

We will define inference rules for _judgements_ of the form: $WellFormed(\texttt{e}, S)$ that says that expression `e` is _WellFormed_ under the set of declared in scope variables $S$.

Constants are well formed under any set $S$. 

$$ \begin{array}{c}
\\
\hline
WellFormed(\texttt{Const(f)}, S) \\
\end{array} \text{(const-rule)} $$

The rules for `True` and `False` are similar.
Identifiers are well formed only if they belong to $S$:

$$\begin{array}{c}
x \in S \\
\hline
WellFormed(\texttt{Ident(x)}, S) \\
\end{array} \text{(ident-rule)} $$

The rules for binary operator `Plus, Minus, Mult, Div,  Geq, Eq, And, Or` are all very similar.

$$ \begin{array}{c}
WellFormed(\texttt{e1}, S) \;\;\; WellFormed(\texttt{e2}, S)\;\;\; T \in \{ \texttt{Plus}, \texttt{Minus}, \texttt{Mult}, \texttt{Div}, \texttt{Geq}, \texttt{Eq}, \texttt{And}, \texttt{Or} \} \\
\hline
WellFormed(\texttt{T(e1, e2)}, S) \\
\end{array} \text{(well-formed-binary-op)} $$

Likewise, for unary operators 

$$ \begin{array}{c}
WellFormed(\texttt{e1}, S) \;\;\; T \in \{ \texttt{Log}, \texttt{Exp}, \texttt{Sine}, \texttt{Cosine}, \texttt{Not}, \texttt{Eq} \} \\
\hline
WellFormed(\texttt{T(e1)}, S) \\
\end{array} \text{(well-formed-unary-op)} $$

The main rule is for let bindings

$$\begin{array}{c}
WellFormed(\texttt{e1}, S) \;\;\; WellFormed(\texttt{e2}, S \cup \{ x\} ) \\
\hline
WellFormed(\texttt{Let(x, e1, e2)}, S) \\
\end{array} \text{(let-rule)} $$

Suppose `e1` is well formed under $S$ and `e2` is well formed under $S$ with the identifier `x` added,
then `let x = e1 in e2` is well formed under $S$, as well. 
- This is confusing when you first see it, but repeat the sentence above and match it with the rule, 
- Notice that the defining expression must be well formed under the original S but the body expression is allowed to be well formed under $S \cup \{ x\} $.
- Once you actualy get it, you can appreciate how delightfully _simple_ it is!

We can also add the rules for function definition:

$$\begin{array}{c}
WellFormed(\texttt{e}, S \cup \{x\}) \\
\hline
WellFormed(\texttt{FunDef(x, e)}, S ) \\
\end{array} \; \text{(fundef-rule)} $$

Finally, function calls are very simple:

$$ \begin{array}{c}
\texttt{f} \in S,\;\;\;\; WellFormed(\texttt{e}, S) \\
\hline
WellFormed(\texttt{f(e)}, S ) \\
\end{array} \text{(well-formed-funcall)} $$

The name of the function `f` better be defined in the scope $S$ and the argument better be OK. Then
we can say that the function call is.

With these rules, we can bravely write our first static check for Lettuce to catch programs that declare variables before use.


In [10]:
def isWellFormedExpression(e: Expr, S: Set[String]): Boolean = e match {
    case Const(_) | True | False  => true
    case Ident(x) => {
        if (S contains x) 
        { true } 
        else 
        { println(s"Error: undeclared identifier: $x");
          false }
        }
    case Plus(e1, e2) => isWellFormedExpression(e1, S) && isWellFormedExpression(e2, S)
    case Minus(e1, e2) => isWellFormedExpression(e1, S) && isWellFormedExpression(e2, S)
    case Mult(e1, e2) => isWellFormedExpression(e1, S) && isWellFormedExpression(e2, S)
    case Div(e1, e2) => isWellFormedExpression(e1, S) && isWellFormedExpression(e2, S)
    case Geq(e1, e2) => isWellFormedExpression(e1, S) && isWellFormedExpression(e2, S)
    case Eq(e1, e2) => isWellFormedExpression(e1, S) && isWellFormedExpression(e2, S)
    case And(e1, e2) => isWellFormedExpression(e1, S) && isWellFormedExpression(e2, S)
    case Or(e1, e2) => isWellFormedExpression(e1, S) && isWellFormedExpression(e2, S)
    case Log(e) => isWellFormedExpression(e, S) 
    case Exp(e) => isWellFormedExpression(e, S) 
    case Sine(e) => isWellFormedExpression(e, S) 
    case Cosine(e) => isWellFormedExpression(e, S) 
    case Not(e) => isWellFormedExpression(e, S) 
    
    case IfThenElse(e1, e2, e3) => isWellFormedExpression(e1, S) && isWellFormedExpression(e2, S) && isWellFormedExpression(e3, S)
    
    case Let(x, e1, e2) => isWellFormedExpression(e1, S) && { 
        val S1 = S + x 
        isWellFormedExpression(e2, S1)
    }
    
    case FunDef(x, e) => isWellFormedExpression(e, S+x)
    
    case FunCall(f, e) => isWellFormedExpression(f, S) && isWellFormedExpression(e, S)
}



def isWellFormedProgram(p: Program ) = p match {
    case TopLevel(e) => isWellFormedExpression(e, Set())
}



defined function isWellFormedExpression
defined function isWellFormedProgram

In [11]:
isWellFormedProgram(p1)

res10: Boolean = true

In [12]:
isWellFormedProgram(p2)

res11: Boolean = true

In [13]:
val p3 = Let("x", Plus(Ident("y"), Ident("z")), Mult(Ident("x"), Ident("y"))) 

p3: Let = Let("x", Plus(Ident("y"), Ident("z")), Mult(Ident("x"), Ident("y")))

In [14]:
isWellFormedProgram(TopLevel(p3))

Error: undeclared identifier: y


res13: Boolean = false

__Q:__ Why does the above program just print one error message about "y" being 
undeclared, and not three error messages that complain about "y" twice and "z" once?



Let's try the program

~~~
let x = x in x * y
~~~

In [15]:
val p4 = Let("x", Ident("x"), Mult(Ident("x"), Ident("y"))) 

p4: Let = Let("x", Ident("x"), Mult(Ident("x"), Ident("y")))

In [16]:
isWellFormedProgram(TopLevel(p4))

Error: undeclared identifier: x


res15: Boolean = false

## Writing an Eval for Lettuce (Without Functions).

Let us try to write an `eval` function for Lettuce. We can work with the existing `eval` function that we already defined for expressions, right?

To avoid tackling too much in one swoop, let us ignore function definitions and function calls for now.

Recall what $\sigma \models \texttt{e} \Downarrow v$ meant? 

Well, let's use less fancy notation and convey exactly the same sentiment using the notation
$$eval(\texttt{e}, \sigma) = v $$ 

- $\sigma$ refers to an environment that maps names of identifiers to their values.
- $\text{domain}(\sigma)$ refers to the domain of $\sigma$.
- $\phi$ will refer to the empty environment in which no identifier is defined.
- If $\sigma$ is an environment, then $\sigma[x \mapsto v]$ is a new environment in which the identifier $x$ is mapped to the value $v$.


What are the values we need to have: 
- Real values belonging to the set $\mathbb{R}$ (or Double precision numbers in scala). We discussed in class what the differences would be.
- Boolean values `true, false` to the set $\mathbb{B}$.
- The special value `error` to the set $\mathbb{Err}$.

The rules are mostly the same, save for a change in notation from $\sigma \models \texttt{e} \Downarrow v$ to 
$eval(\texttt{e}, \sigma) = v$.

$$\begin{array}{c}
\\
\hline
eval(\texttt{Const(v)}, \sigma) = v \\
\end{array} \text{(const-rule)} $$

$$\begin{array}{c}
x \in \text{domain}(\sigma) \\
\hline
eval(\texttt{Ident(x)}, \sigma) = \sigma(\texttt{x}) \\
\end{array} \text{(ident-ok-rule)}\ \;\;\; \begin{array}{c}
x \not\in \text{domain}(\sigma) \\
\hline
eval(\texttt{Ident(x)}, \sigma) = \mathbf{error} \\
\end{array} \text{(ident-nok-rule)} $$


Let us read the (ident-ok-rule) aloud. It says that if a _variable x belongs to the domain of $\sigma$_ then
_the expression `Ident(x)` evaluates to $\sigma(x)$_. Makes sense?
<font color="red"> Exercise: read the (ident-nok-rule) aloud in your own words. </font> 

<font color="red"> Q: </font> If we checked that the expression is well-formed, then will we need (ident-nok-rule)?

Rules for `Plus, Minus, Mult`. 

$$\begin{array}{c}
eval(\texttt{e1}, \sigma) = v_1,\; \; eval(\texttt{e2}, \sigma) = v_2,\ \ v_1 \in \mathbb{R},\ \ v_2 \in \mathbb{R}, \; \; \texttt{T} \in \{ \texttt{Plus, Minus, Mult} \}  \\
\hline
eval(\texttt{T(e1, e2)}, \sigma) = f_T(v_1, v_2) \\
\end{array} \text{(arith-binop-ok-rule)}$$

Note that the rule refers to $f_T$ for the operator $T$. 
Define these as $f_{Plus}(x,y) = x + y$, $f_{Mult} = x * y$ and $f_{Minus} (x, y) = x - y$.

The `Div` operator needs special handling in exactly the same way we showed in our previous lectures.
<font color="red"> Write down the rules for `Div` as an exercise </font>


We can now handle the case when subexpressions of arithmetic expressions yield a non real value (these can be Booleans or even error).

$$\begin{array}{c}
eval(\texttt{e1}, \sigma) = v_1,\; \ v_1 \not\in \mathbb{R},\ ; \; \texttt{T} \in \{ \texttt{Plus, Minus, Mult, Div} \}  \\
\hline
eval(\texttt{T(e1, e2)}, \sigma) = \mathbf{error} \\
\end{array} \text{(arith-binop-type-mismatch-rule-1)}\;\;
\begin{array}{c}
eval(\texttt{e1}, \sigma) = v_1,\; eval(\texttt{e2}, \sigma) = v_2,\; \ v_1 \in \mathbb{R},\; \; v_2 \not\in \mathbb{R},\ ; \; \texttt{T} \in \{ \texttt{Plus, Minus, Mult, Div} \}  \\
\hline
eval(\texttt{T(e1, e2)}, \sigma) = \mathbf{error} \\
\end{array} \text{(arith-binop-type-mismatch-rule-2)}
$$

Unary arithmetic operators are also handled in the same way as in the previous lectures.
$$\begin{array}{c}
eval(\texttt{e}, \sigma) = v,\;\; v \in \mathbb{R} \; \; \texttt{T} \in \{ \texttt{Exp, Sine, Cosine} \}  \\
\hline
eval(\texttt{T(e)}, \sigma) = f_T(v) \\
\end{array} \text{(arith-unop-ok-rule)}$$

Wherein $f_{Exp}(x) = e^x,\ f_{Sine}(x) = \sin(x), f_{Cosine}(x) = \cos(x)$.
Log needs special rule to deal with argument being positive vs. non-positive.

$$\begin{array}{c}
eval(\texttt{e}, \sigma) = v,\;\; v \in \mathbb{R},\ v > 0  \\
\hline
eval(\texttt{Log(e)}, \sigma) = \log(v) \\
\end{array} \text{(log-ok-rule)}\;\;\;
\begin{array}{c}
eval(\texttt{e}, \sigma) = v,\;\; v \in \mathbb{R},\ v \leq 0  \\
\hline
eval(\texttt{Log(e)}, \sigma) = \mathbf{error} \\
\end{array} \text{(log-nok-rule)}$$

$$\begin{array}{c}
eval(\texttt{e}, \sigma) = v,\; \ v \not\in \mathbb{R},\ ; \; \texttt{T} \in \{ \texttt{Log, Sine, Cosine, Exp} \}  \\
\hline
eval(\texttt{T(e)}, \sigma) = \mathbf{error} \\
\end{array} \text{(arith-unop-type-mismatch-rule)}$$

#### Rules for Boolean Operators

$$\begin{array}{c} 
\\
\hline
eval(\texttt{True}, \sigma) = true \\
\end{array} \text{(true rule)} \;\;\;\;
\begin{array}{c} 
\\
\hline
eval(\texttt{False}, \sigma) = false \\
\end{array}\text{(false rule)}
$$

`And` and `Or` can be short circuited. We will write the rules for `And` and note that
the rules of `Or` and `Not` are similar.

$$\begin{array}{c} 
eval(\texttt{e1}, \sigma) = false\\
\hline
eval(\texttt{And}(e1, e2), \sigma) = false\\
\end{array} \text{(and-arg-1-ok-rule)} \;\;\;\;
\begin{array}{c} 
eval(\texttt{e1}, \sigma) = v_1,\ v_1 \not\in \mathbb{B} \\
\hline
eval(\texttt{And}(e1, e2), \sigma) = \mathbf{error}\\
\end{array}\text{(and-arg-1-nok-rule)}
$$

$$\begin{array}{c} 
eval(\texttt{e1}, \sigma) = true\;\; eval(\texttt{e2}, \sigma) = v_2,\ \;\; v_2 \in \mathbb{B}\\
\hline
eval(\texttt{And}(e1, e2), \sigma) = v_2\\
\end{array} \text{(and-arg-2-ok-rule)} \;\;\;\;
\begin{array}{c} 
eval(\texttt{e1}, \sigma) = true,\ eval(\texttt{e2}, \sigma) = v_2,\  \;\; v_2 \not\in \mathbb{B} \\
\hline
eval(\texttt{And}(e1, e2), \sigma) = \mathbf{error}\\
\end{array}\text{(and-arg-2-nok-rule)}
$$

<font color="red"> As an exercise write rules for  `Geq, Eq` </font> . They are going to be very similar to those written thus far but without short circuit semantics for regular case and with short circuit for errors.

#### Rule for Let Binding

$$\begin{array}{c} 
eval(\texttt{e1}, \sigma) = v_1,\ v_1 \not= \mathbf{error}\;\; eval(\texttt{e2},   \color{red}{\sigma[x \mapsto v_1]}) = v_2,\ \;\; v_2 \not= \mathbf{error}\\
\hline
eval(\texttt{Let(x,e1, e2)}, \sigma) = v_2\\
\end{array} \text{(let-binding-ok)} $$

The most important part of the rule above is notice that $\texttt{e2}$ is being evaluated under
$\color{red}{\sigma[ x \mapsto v_1]}$, which is the environment $\sigma$ extended with $x$ bound to $v_1$.

$$\begin{array}{c} 
eval(\texttt{e1}, \sigma) =  \mathbf{error}\\
\hline
eval(\texttt{Let(x,e1, e2)}, \sigma) = \mathbf{error}\\
\end{array} \text{(let-binding-nok-1)}
\begin{array}{c} 
eval(\texttt{e1}, \sigma) =  v_1,\; v_1 \not= \mathbf{error}\; eval(\texttt{e2}, \sigma[x \mapsto v_1]) =  \mathbf{error}\\
\hline
eval(\texttt{Let(x,e1, e2)}, \sigma) = \mathbf{error}\\
\end{array} \text{(let-binding-nok-2)}$$


Given these rules, the final rule is for `TopLevel`. We recall that $\phi$ denotes the empty environment.

$$\begin{array}{c}
eval(\texttt{e}, \phi) = v \\
\hline
eval(\texttt{TopLevel(e)}, \phi) = v \\
\end{array} \text{(toplevel-eval)} $$

### Implementing the Interpreter 

First let us implement what we will need as part of the value class.

In [25]:
/* 1. Define the values */
sealed trait Value 
case class NumValue(d: Double) extends Value
case class BoolValue(b: Boolean) extends Value
case object ErrorValue extends Value

/*2. Operators on values */

def valueToNumber(v: Value): Double = v match {
    case NumValue(d) => d
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a number")
}

def valueToBoolean(v: Value): Boolean = v match {
    case BoolValue(b) => b
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a boolean")
}


defined trait Value
defined class NumValue
defined class BoolValue
defined object ErrorValue
defined function valueToNumber
defined function valueToBoolean

In [26]:
def evalExpr(e: Expr, env: Map[String, Value]): Value =  {
    
    /* Method to deal with binary arithmetic operations */
    
    def applyArith2 (e1: Expr, e2: Expr) (fun: (Double , Double) => Double) = {
        val v1 = valueToNumber(evalExpr(e1, env))
        val v2 = valueToNumber(evalExpr(e2, env))
        val v3 = fun(v1, v2)
        NumValue(v3)
    }  /* -- We have deliberately curried the method --*/
    
    /* Helper method to deal with unary arithmetic */
    def applyArith1(e: Expr) (fun: Double => Double) = {
        val v = valueToNumber(evalExpr(e, env))
        val v1 = fun(v)
        NumValue(v1)
    }
    
    /* Helper method to deal with comparison operators */
    def applyComp(e1: Expr, e2: Expr) (fun: (Double, Double) => Boolean) = {
        val v1 = valueToNumber(evalExpr(e1, env))
        val v2 = valueToNumber(evalExpr(e2, env))
        val v3 = fun(v1, v2)
        BoolValue(v3)
    }
    
   
    e match {
        case Const(f) => NumValue(f)
        
        case Ident(x) => {
            if (env contains x) 
                env(x)
            else 
                throw new IllegalArgumentException(s"Undefined identifier $x")
        }
    
        case True => BoolValue(true)
    
        case False => BoolValue(false)
    
        case Plus(e1, e2) => applyArith2 (e1, e2) ( _ + _ )
    
        case Minus(e1, e2) => applyArith2(e1, e2) ( _ - _ )
    
        case Mult(e1, e2) =>  applyArith2(e1, e2) (_ * _)
    
        case Div(e1, e2) => applyArith2(e1, e2) {
            case (_, 0.0) => throw new IllegalArgumentException(s"Divide by zero error in divisor ${e2}")
            case (v1, v2) => v1/v2
        }
    
        case Log(e1) => applyArith1(e1) {
            case v if v > 0.0 => math.log(v)
            case v => throw new IllegalArgumentException(s"Log of a negative number ${e} evaluates to ${v}!")
        }
    
        case Exp(e1) => applyArith1(e1)(math.exp)
    
        case Sine(e1) => applyArith1(e1)(math.sin)
    
        case Cosine(e1) => applyArith1(e1)(math.cos)
    
        case Geq(e1, e2) => applyComp(e1, e2)(_ >= _)
    
        case Eq(e1, e2) => applyComp(e1, e2)(_ == _)
    
        case And(e1, e2) => { /* Short circuit eval of And */
            val v1 = evalExpr(e1, env)
            v1 match {
                case BoolValue(false) => BoolValue(false)
                case BoolValue(true) => {
                    val v2 = evalExpr(e2, env)
                    v2 match {
                        case BoolValue(_) => v2
                        case _ => throw new IllegalArgumentException(
                            s"And of a boolean and non-boolean expr: ${e2} which evaluated to ${v2}")       
                    }
                }
                case _ => throw new IllegalArgumentException(s"And of a non-boolean expr: ${e1} which evaluted to ${v1}")
            }
        }
    
        case Or(e1, e2) => { /* Short circuit eval of OR*/
            val v1 = evalExpr(e1, env)
            v1 match {
                case BoolValue(true) => BoolValue(true)
                case BoolValue(false) => {
                    val v2 = evalExpr(e2, env)
                    v2 match {
                        case BoolValue(_) => v2
                        case _ => throw new IllegalArgumentException(
                            s"Or of a boolean and non-boolean expr: ${e2} which evaluated to ${v2}")       
                    }
                }
                case _ => throw new IllegalArgumentException(s"Or of a non-boolean expr: ${e1} which evaluted to ${v1}")
            }
        }
    
        case Not(e1) => {
            val v = evalExpr(e1, env)
            v match {
                case BoolValue(b) => BoolValue(!b)
                case _ => throw new IllegalArgumentException(s"Not of a non-boolean expr: ${e} which evaluated to ${v}")
            }
        }
    
        case IfThenElse(e1, e2, e3) => {
            val v = evalExpr(e1, env)
            v match {
                case BoolValue(true) => evalExpr(e2, env)
                case BoolValue(false) => evalExpr(e3, env)
                case _ => throw new IllegalArgumentException(s"If-then-else condition expr: ${e1} is non-boolean -- evaluates to ${v}")
            }
        }
    
        case Let(x, e1, e2) => {
            val v1 = evalExpr(e1, env)  // eval e1
            val env2 = env + (x -> v1) // create a new extended env
            evalExpr(e2, env2) // eval e2 under that.
        }
    
        case _:FunDef => throw new IllegalArgumentException("Function definitions not yet handled in this interpreter.")
    
        case _:FunCall => throw new IllegalArgumentException("Function calls not yet handled in this interpreter.")
    }
}

def evalProgram(p: Program) = {
    val m: Map[String, Value] = Map[String,Value]()
    p match { 
        case TopLevel(e) => {
            try {
                evalExpr(e, m)
            } catch {
                case e: IllegalArgumentException => {
                    println(s"Error: $e")
                    ErrorValue
                }
            }
        }
    }
}

defined function evalExpr
defined function evalProgram

In [19]:
print(p1)
evalProgram(p1)

TopLevel(Let(x,Plus(Const(10.0),Const(15.0)),Let(y,Geq(Ident(x),Const(25.0)),IfThenElse(Ident(y),Ident(x),Minus(Ident(x),Const(35.0))))))

res18_1: Value = NumValue(25.0)

In [20]:
print(p4)
evalProgram(TopLevel(p4))

Let(x,Ident(x),Mult(Ident(x),Ident(y)))Error: java.lang.IllegalArgumentException: Undefined identifier x


res19_1: Value = ErrorValue

In [21]:
val p5 = TopLevel(Let("x", Const(3.0), Mult(Ident("x"), Ident("x"))))

p5: TopLevel = TopLevel(Let("x", Const(3.0), Mult(Ident("x"), Ident("x"))))

In [22]:
evalProgram(p5)

res21: Value = NumValue(9.0)

In [23]:
val p6 = TopLevel(Let("x", Const(3.0), Geq(Mult(Ident("x"), Ident("x")), Ident("x"))))

p6: TopLevel = TopLevel(
  Let("x", Const(3.0), Geq(Mult(Ident("x"), Ident("x")), Ident("x")))
)

In [24]:
evalProgram(p6)

res23: Value = BoolValue(true)